# I. US County Race Population

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime  


def diagnose_Rate(T, **kwargs):
    info_list = []
    for col in T.columns:
        if col in ['Rate_Black', 'Rate_White', 'Rate_Parity']:
            d = {'name': col}
            for k, v in kwargs.items():
                d[k] = v
                
            for k,v in T[col].describe().to_dict().items():
                d[k] = v
            info_list.append(d)

    Report = pd.DataFrame(info_list)
    return Report



In [2]:
path = 'DataMerge/Data/C-AnnualCountyPopulation/US_COUNTY_POPULATION.csv'
US_COUNTY_POPULATION = pd.read_csv(path)
US_COUNTY_POPULATION.head()

,GEOID,State,County,Geographic Area Name,Total_Whole,Total_White,Total_Black,Total_AIndA,Total_Asian,Total_WhiteC,Total_BlackC,Total_AIndAC,Total_AsianC
0,1001,Alabama,Autauga,"Autauga, Alabama",55869,42607,11237,266,656,43563,11784,575,950
1,1003,Alabama,Baldwin,"Baldwin, Alabama",223234,195198,19593,1742,2380,199103,21354,3603,3351
2,1005,Alabama,Barbour,"Barbour, Alabama",24686,12134,11906,170,116,12380,12117,297,152
3,1007,Alabama,Bibb,"Bibb, Alabama",22394,17191,4763,103,48,17420,4907,207,97
4,1009,Alabama,Blount,"Blount, Alabama",57826,55412,978,370,185,56167,1276,820,312


In [3]:
STATE_VACCINE = {}

In [5]:
print("Enter the date of collecting data (Today's data in YYYY-MM-DD):")
Today_Date = input()

assert Today_Date == str(datetime.now().date())

print('Hello, ' + Today_Date)


Enter the date of collecting data (Today's data in YYYY-MM-DD):


 2021-04-26


Hello, 2021-04-26


# II. API States

## A. Illinois* (Done)


https://www.dph.illinois.gov/covid19/vaccinedata?county=Illinois


* Only Number


* Admin-Count (People at least One)

* Full-Count (Fully Covered)


In [6]:
from datetime import datetime 
import numpy as np

import requests
import time



In [7]:
state = 'Illinois'


def get_Illinois_Raw_and_postprocess(US_COUNTY_POPULATION):
    

    il_county_list = [i for i in US_COUNTY_POPULATION[US_COUNTY_POPULATION['State'] == 'Illinois']['County'].values]

    il_county_list.sort()

    L = []
    error_list = []
    for c in il_county_list:
        # c = 'Mchenry'
        api_url = 'https://idph.illinois.gov/DPHPublicInformation/api/covidvaccine/getVaccineAdministrationDemos?countyname={}'.format(c)

        resp = requests.get(api_url)# .to_json()
        data = resp.json() # resp
        if sum([len(v) for k, v in data.items()]) == 0:
            error_list.append(c)
            print(c)
            continue

        time.sleep(0.1)
        try:
            D = {}
            D['County'] = c
            for d in data['Race']:
                D[d['Race'] + '-AdminCount'] = d['AdministeredCount']
                D[d['Race'] + '-FullyVaccn'] = d['PersonsFullyVaccinated']
                date = d['Report_Date']

            D['Date'] = date.split('T')[0]
            L.append(D)

        except:
            print('No Data')
            print(c)

    IL_County_Vaccine_Race = pd.DataFrame(L)
    
    return IL_County_Vaccine_Race




IL_County_Vaccine_Race = get_Illinois_Raw_and_postprocess(US_COUNTY_POPULATION)
# IL_County_Vaccine_Race.to_csv('FinalResult/Raw_Illinois.csv')
print(IL_County_Vaccine_Race.shape)
IL_County_Vaccine_Race.head()

(102, 18)


,County,Hispanic or Latino-AdminCount,Hispanic or Latino-FullyVaccn,Asian-AdminCount,Asian-FullyVaccn,Other race-AdminCount,Other race-FullyVaccn,Black or African-American-AdminCount,Black or African-American-FullyVaccn,White-AdminCount,White-FullyVaccn,American Indian or Alaska Nati-AdminCount,American Indian or Alaska Nati-FullyVaccn,Native Hawaiian or Other Pacif-AdminCount,Native Hawaiian or Other Pacif-FullyVaccn,Unknown-AdminCount,Unknown-FullyVaccn,Date
0,Adams,489,263,342.0,180.0,84,50,1095,604,39845,21234,54.0,27.0,53.0,26.0,803,386,2021-04-25
1,Alexander,8,3,NaN,NaN,7,3,467,211,814,376,5.0,1.0,2.0,1.0,100,48,2021-04-25
2,Bond,134,51,39.0,17.0,106,73,143,60,8420,3974,1.0,1.0,9.0,3.0,267,122,2021-04-25
3,Boone,3853,1213,373.0,152.0,503,219,464,188,24937,10703,113.0,53.0,39.0,16.0,1618,663,2021-04-25
4,Brown,10,5,6.0,3.0,1,1,22,12,2634,1281,6.0,3.0,NaN,NaN,59,26,2021-04-25


In [8]:


def standardize_Illinois(IL_County_Vaccine_Race, US_COUNTY_POPULATION):
    
    state = 'Illinois'

    new_L = []


    for idx, row in IL_County_Vaccine_Race.iterrows(): # change this
        d = row.to_dict()

        new_d = {}


        # GEO Information
        new_d['State'] = state
        new_d['County'] = d['County']


        RenameCounties = {
            # 'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



        # At Least One
        new_d['Numb_White'] = d['White-AdminCount']
        new_d['Rate_White'] = np.nan

        new_d['Numb_Black'] = d['Black or African-American-AdminCount']
        new_d['Rate_Black'] = np.nan

        new_d['Numb_Asian'] = d['Asian-AdminCount']
        new_d['Rate_Asian'] = np.nan

        new_d['Numb_AIndA'] = d['American Indian or Alaska Nati-AdminCount']
        new_d['Rate_AIndA'] = np.nan

        new_d['Numb_Other'] = d['Other race-AdminCount']
        
       

        new_d['Numb_Hispa'] = d['Hispanic or Latino-AdminCount']
        

        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)

    
    Data = pd.DataFrame(new_L)
    
    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)


    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian', 'AIndA']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate


        Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race + 'C'] = Rate

        
    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    return T
        
        


T = standardize_Illinois(IL_County_Vaccine_Race, US_COUNTY_POPULATION)
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

(102, 27)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_Hispa,Numb_Other,Numb_White,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Illinois,Adams,"Adams, Illinois",2021-04-26,54.0,342.0,1095,489,84,39845,...,0.642755,190,395,615,830,2557,3554,60698,61991,65435
1,Illinois,Alexander,"Alexander, Illinois",2021-04-26,5.0,NaN,467,8,7,814,...,0.210119,26,62,17,36,1830,1917,3740,3874,5761
2,Illinois,Bond,"Bond, Illinois",2021-04-26,1.0,39.0,143,134,106,8420,...,0.557136,110,183,134,187,1041,1195,14874,15113,16426
3,Illinois,Boone,"Boone, Illinois",2021-04-26,113.0,373.0,464,3853,503,24937,...,0.491738,420,1006,785,1052,1503,1984,49530,50712,53544
4,Illinois,Brown,"Brown, Illinois",2021-04-26,6.0,6.0,22,10,1,2634,...,0.505372,87,107,21,43,1242,1276,5165,5212,6578


In [9]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,102.0,0.235599,0.132264,0.016427,0.146255,0.235328,0.305950,0.990991
1,Rate_White,102.0,0.549531,0.101545,0.217647,0.490342,0.556007,0.622815,0.824411
2,Rate_Parity,102.0,0.313932,0.141583,-0.395382,0.250741,0.328689,0.411728,0.580483


# III. Table States

## B. Texas* (Done)

download from: 
https://www.dshs.texas.gov/immunize/covid19/COVID-19-Vaccine-Data-by-County.xls


which is from: https://www.dshs.texas.gov/coronavirus/immunize/vaccine.aspx


Only Number, No Rate.


Three conceptions:

* Administered, 

* Vaccinated

* Fully Vaccinated

In [10]:
def get_Texas_Raw():
    TEXAS_URL = 'https://www.dshs.texas.gov/immunize/covid19/COVID-19-Vaccine-Data-by-County.xls'
    RawData = pd.read_excel(TEXAS_URL, sheet_name='By County, Race')
    return RawData


RawData = get_Texas_Raw()
RawData.columns = [i.strip() for i in RawData.columns]
RawData

,County Name,Race/Ethnicity,People Vaccinated with at least One Dose,People Fully Vaccinated,Doses Administered
0,Anderson,Asian,122,70,184
1,Anderson,Black,1417,753,2019
2,Anderson,Hispanic,984,527,1408
3,Anderson,Other,694,331,988
4,Anderson,Unknown,946,628,1436
...,...,...,...,...,...
1509,Other,Hispanic,29772,14289,42203
1510,Other,Other,8967,5084,13527
1511,Other,Unknown,13826,7763,20636
1512,Other,White,53808,29823,79747


In [11]:
def postprocess_Texas_Raw(RawData):
    RawDataGroups = RawData.groupby('County Name')
    L = []
    for county, record in RawDataGroups:
        D = {}

        D['County'] = county

        for idx, row in record.iterrows():
            d = row.to_dict()
            # print(d)
            race = d['Race/Ethnicity'] 

            D[race+'_DoesesAdmin'] = d['Doses Administered']
            D[race+'_Vaccinated']  = d['People Vaccinated with at least One Dose']
            D[race+'_FullVacc']     = d['People Fully Vaccinated']
        L.append(D)


    Texas_County_Vaccine_Race = pd.DataFrame(L)
    
    return Texas_County_Vaccine_Race


Texas_County_Vaccine_Race = postprocess_Texas_Raw(RawData)
# Texas_County_Vaccine_Race.to_csv('FinalResult/Post_Texas.csv')
print(Texas_County_Vaccine_Race.shape)
Texas_County_Vaccine_Race.head() # 254 counties + 1 total

(256, 22)


,County,Asian_DoesesAdmin,Asian_Vaccinated,Asian_FullVacc,Black_DoesesAdmin,Black_Vaccinated,Black_FullVacc,Hispanic_DoesesAdmin,Hispanic_Vaccinated,Hispanic_FullVacc,...,Other_FullVacc,Unknown_DoesesAdmin,Unknown_Vaccinated,Unknown_FullVacc,White_DoesesAdmin,White_Vaccinated,White_FullVacc,Total_DoesesAdmin,Total_Vaccinated,Total_FullVacc
0,Anderson,184.0,122.0,70.0,2019.0,1417.0,753.0,1408.0,984.0,527.0,...,331.0,1436.0,946.0,628.0,9787.0,6131.0,4373.0,NaN,NaN,NaN
1,Andrews,48.0,28.0,20.0,72.0,39.0,33.0,4187.0,2263.0,1935.0,...,96.0,2250.0,1271.0,1121.0,3651.0,2026.0,1656.0,NaN,NaN,NaN
2,Angelina,438.0,292.0,157.0,4547.0,2840.0,1906.0,5977.0,4094.0,2107.0,...,797.0,7325.0,3940.0,3466.0,19657.0,12060.0,8515.0,NaN,NaN,NaN
3,Aransas,313.0,188.0,132.0,88.0,56.0,35.0,2613.0,1566.0,1173.0,...,941.0,2310.0,1246.0,1076.0,10736.0,6254.0,4854.0,NaN,NaN,NaN
4,Archer,27.0,22.0,14.0,89.0,84.0,36.0,196.0,138.0,94.0,...,422.0,1068.0,569.0,509.0,2849.0,1674.0,1423.0,NaN,NaN,NaN


In [12]:
import numpy as np
def standardize_Texas(Texas_County_Vaccine_Race, US_COUNTY_POPULATION):
    state = 'Texas'

    new_L = []


    for idx, row in Texas_County_Vaccine_Race.iterrows():
        d = row.to_dict()

        new_d = {}


        new_d['State'] = state
        new_d['County'] = d['County']

        RenameCounties = {
            # 'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

        new_d['Numb_White'] = d['White_Vaccinated']
        new_d['Rate_White'] = np.nan

        new_d['Numb_Black'] = d['Black_Vaccinated']
        new_d['Rate_Black'] = np.nan

        new_d['Numb_Asian'] = d['Asian_Vaccinated']
        new_d['Rate_Asian'] = np.nan

        new_d['Numb_AIndA'] = np.nan
        new_d['Rate_AIndA'] = np.nan

        new_d['Numb_Other'] = d['Other_Vaccinated']
        new_d['Rate_Other'] = np.nan


        new_d['Numb_NaHis'] = np.nan
        new_d['Rate_NaHis'] = np.nan

        new_d['Numb_Hispa'] = d['Hispanic_Vaccinated']
        new_d['Rate_Hispa'] = np.nan



        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)
    # StateCountyVaccine[state] = Data
    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)



    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian', 'AIndA']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate


    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    
    return T



T = standardize_Texas(Texas_County_Vaccine_Race, US_COUNTY_POPULATION)

state = 'Texas'
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

Not included Counties ['Other, Texas', 'Grand Total, Texas']
(254, 27)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_Hispa,Numb_NaHis,Numb_Other,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Texas,Anderson,"Anderson, Texas",2021-04-26,NaN,122.0,1417.0,984.0,NaN,694.0,...,0.142465,428,747,538,757,12628,13200,43035,43927,57735
1,Texas,Andrews,"Andrews, Texas",2021-04-26,NaN,28.0,39.0,2263.0,NaN,124.0,...,0.114846,274,392,130,171,382,502,17641,17900,18705
2,Texas,Angelina,"Angelina, Texas",2021-04-26,NaN,292.0,2840.0,4094.0,NaN,1411.0,...,0.171731,665,1040,923,1166,13595,14256,70226,71385,86715
3,Texas,Aransas,"Aransas, Texas",2021-04-26,NaN,188.0,56.0,1566.0,NaN,1168.0,...,0.286342,308,542,491,564,406,554,21841,22252,23510
4,Texas,Archer,"Archer, Texas",2021-04-26,NaN,22.0,84.0,138.0,NaN,504.0,...,0.205702,131,200,41,66,96,153,8138,8264,8553


In [13]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,248.0,0.160710,0.205649,0.012821,0.085451,0.146556,0.195664,3.000000
1,Rate_White,254.0,0.163468,0.062662,0.020272,0.124195,0.168932,0.201506,0.333989
2,Rate_Parity,248.0,0.003619,0.203002,-2.838828,-0.025965,0.012477,0.056014,0.233733


## B. Pennsylvania* (Done)

67 counties

Ethnicity
https://data.pa.gov/Health/COVID-19-Vaccinations-by-Ethnicity-Current-County-/7ruj-m7k6 

Race
https://data.pa.gov/Health/COVID-19-Vaccinations-by-Race-Current-County-Healt/x5z9-57ub


Only Number

In [14]:

Penn_Ethn_URL = 'https://data.pa.gov/api/views/7ruj-m7k6/rows.csv?accessType=DOWNLOAD'
Penn_Race_URL = 'https://data.pa.gov/api/views/x5z9-57ub/rows.csv?accessType=DOWNLOAD'

Ethn = pd.read_csv(Penn_Ethn_URL).rename(columns = {'Ethnicity':'Item', 'County Name': 'County'})
Race = pd.read_csv(Penn_Race_URL).rename(columns = {'Race':'Item'})


RawData = pd.concat([Race, Ethn])

RawData.columns


Index(['County Name', 'Partially Covered African American',
       'Partially Covered Asian', 'Partially Covered Native American',
       'Partially Covered Pacific Islander',
       'Partially Covered Multiple Other', 'Partially Covered White',
       'Partially Covered  Unknown', 'Fully Covered African American',
       'Fully Covered Asian', 'Fully Covered Native American',
       'Fully Covered Pacific Islander', 'Fully Covered Multiple Other',
       'Fully Covered White', 'Fully Covered Unknown', 'County',
       'Partially Covered Hispanic', 'Partially Covered Not Hispanic',
       'Partially Covered Unknown', 'Fully Covered Hispanic',
       'Fully Covered Not Hispanic'],
      dtype='object')

In [15]:
Race.head()

,County Name,Partially Covered African American,Partially Covered Asian,Partially Covered Native American,Partially Covered Pacific Islander,Partially Covered Multiple Other,Partially Covered White,Partially Covered Unknown,Fully Covered African American,Fully Covered Asian,Fully Covered Native American,Fully Covered Pacific Islander,Fully Covered Multiple Other,Fully Covered White,Fully Covered Unknown
0,LUZERNE,845.0,76.0,8.0,100.0,2196,29055,4067,1236.0,51.0,18.0,129.0,3380,77569,7273
1,CUMBERLAND,844.0,93.0,9.0,74.0,3235,29324,11133,1246.0,136.0,19.0,98.0,5766,44532,5971
2,CAMBRIA,216.0,9.0,NaN,12.0,786,9697,1176,288.0,29.0,NaN,14.0,2268,30428,4258
3,PERRY,19.0,NaN,NaN,NaN,400,4208,2759,19.0,NaN,NaN,NaN,788,6922,453
4,JEFFERSON,5.0,NaN,NaN,NaN,67,2514,577,6.0,9.0,NaN,NaN,340,10070,1959


In [16]:
import pandas as pd

Pennsylvania_County_Vaccine_Race =  Race                         
Pennsylvania_County_Vaccine_Race

,County Name,Partially Covered African American,Partially Covered Asian,Partially Covered Native American,Partially Covered Pacific Islander,Partially Covered Multiple Other,Partially Covered White,Partially Covered Unknown,Fully Covered African American,Fully Covered Asian,Fully Covered Native American,Fully Covered Pacific Islander,Fully Covered Multiple Other,Fully Covered White,Fully Covered Unknown
0,LUZERNE,845.0,76.0,8.0,100.0,2196,29055,4067,1236.0,51.0,18.0,129.0,3380,77569,7273
1,CUMBERLAND,844.0,93.0,9.0,74.0,3235,29324,11133,1246.0,136.0,19.0,98.0,5766,44532,5971
2,CAMBRIA,216.0,9.0,NaN,12.0,786,9697,1176,288.0,29.0,NaN,14.0,2268,30428,4258
3,PERRY,19.0,NaN,NaN,NaN,400,4208,2759,19.0,NaN,NaN,NaN,788,6922,453
4,JEFFERSON,5.0,NaN,NaN,NaN,67,2514,577,6.0,9.0,NaN,NaN,340,10070,1959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,MONROE,2346.0,14.0,NaN,41.0,2172,15326,2185,2162.0,10.0,NaN,27.0,2514,26045,1981
64,MIFFLIN,18.0,NaN,NaN,NaN,233,2872,172,42.0,NaN,NaN,6.0,787,10003,338
65,COLUMBIA,65.0,10.0,NaN,13.0,200,5970,278,110.0,8.0,NaN,16.0,995,17063,580
66,VENANGO,27.0,NaN,NaN,NaN,206,2585,255,49.0,5.0,NaN,16.0,1957,8462,989


In [17]:
def standardize_Penn(Pennsylvania_County_Vaccine_Race, US_COUNTY_POPULATION):

    state = 'Pennsylvania' ## 

    new_L = []

    for idx, row in Pennsylvania_County_Vaccine_Race.iterrows(): ## 
        d = row.to_dict()

        new_d = {}
        new_d['State'] = state
        new_d['County'] = d['County Name'].lower().capitalize()

        RenameCounties = {
            'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



        new_d['Numb_White'] = d['Partially Covered White']                  + d['Fully Covered White']   
        
        new_d['Numb_Black'] = d['Partially Covered African American']       + d['Fully Covered African American']
        
        new_d['Numb_Asian'] = d['Partially Covered Asian']                  + d['Fully Covered Asian']                  
         


        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)
    
    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)

    
    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')
    print(T.columns)
    for Race in ['White', 'Black', 'Asian']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate



        
    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    
    return Data, T

In [18]:
Data, T = standardize_Penn(Pennsylvania_County_Vaccine_Race, US_COUNTY_POPULATION)
state = 'Pennsylvania'
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

Not included Counties ['Out-of-state, Pennsylvania']
Index(['State', 'County', 'Geographic Area Name', 'Numb_White', 'Numb_Black',
       'Numb_Asian', 'Date', 'Total_Whole', 'Total_White', 'Total_Black',
       'Total_AIndA', 'Total_Asian', 'Total_WhiteC', 'Total_BlackC',
       'Total_AIndAC', 'Total_AsianC'],
      dtype='object')
(67, 19)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Pennsylvania,Luzerne,"Luzerne, Pennsylvania",2021-04-26,127.0,2081.0,106624,0.028153,0.099081,0.376312,2080,3575,4511,5657,21003,25260,283339,289123,317417
1,Pennsylvania,Cumberland,"Cumberland, Pennsylvania",2021-04-26,229.0,2090.0,73856,0.018901,0.175970,0.331171,595,1688,12116,13967,11877,15019,223015,228310,253370
2,Pennsylvania,Cambria,"Cambria, Pennsylvania",2021-04-26,38.0,504.0,40125,0.056972,0.104068,0.328194,206,587,667,1007,4843,6403,122260,124293,130192
3,Pennsylvania,Perry,"Perry, Pennsylvania",2021-04-26,NaN,38.0,11130,NaN,0.070896,0.248626,146,362,225,330,536,844,44766,45293,46272
4,Pennsylvania,Jefferson,"Jefferson, Pennsylvania",2021-04-26,NaN,11.0,12584,NaN,0.046218,0.296199,118,308,128,193,238,442,42485,42910,43425


In [19]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,62.0,0.160348,0.073620,0.024366,0.111228,0.149346,0.204549,0.394639
1,Rate_White,67.0,0.308591,0.080182,0.057450,0.255094,0.301760,0.355413,0.497676
2,Rate_Parity,62.0,0.144352,0.073295,-0.117550,0.107750,0.148406,0.181888,0.287063


In [20]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,62.0,0.160348,0.073620,0.024366,0.111228,0.149346,0.204549,0.394639
1,Rate_White,67.0,0.308591,0.080182,0.057450,0.255094,0.301760,0.355413,0.497676
2,Rate_Parity,62.0,0.144352,0.073295,-0.117550,0.107750,0.148406,0.181888,0.287063


## B. Indiana* (Done)

https://hub.mph.in.gov/dataset/covid-19-vaccinations-demographics-by-county-and-district/resource/82d99020-093f-41ac-95c7-d3c335b8c2ba

In [21]:
# path = 'TMP/Indiana_Mar26_county-vaccination-demographics.xlsx'
# path = 'TMP/Indiana/Indiana_April6_county-vaccination-demographics.xlsx'

url = 'https://hub.mph.in.gov/dataset/145a43b2-28e5-4bf1-ad86-123d07fddb55/resource/82d99020-093f-41ac-95c7-d3c335b8c2ba/download/county-vaccination-demographics.xlsx'

RaceData = pd.read_excel(url, sheet_name = 'Race').rename(columns = {"race":"item"})
EthnicityData = pd.read_excel(url, sheet_name = 'Ethnicity').rename(columns = {"ethnicity":"item"})
Data = pd.concat([RaceData, EthnicityData])
Data.head()

,county,region,fips,item,first_dose_administered,second_dose_administered,single_dose_administered,all_doses_administered,fully_vaccinated,new_first_dose_administered,new_second_dose_administered,new_single_dose_administered,new_all_doses_administered,new_fully_vaccinated,current_as_of
0,Adams,3,18001,Asian,46,28,Suppressed,74,28,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,4/26/2021 5:00
1,Adams,3,18001,Black or African American,16,11,Suppressed,31,15,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,4/26/2021 5:00
2,Adams,3,18001,Other Race,176,133,33,342,166,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,4/26/2021 5:00
3,Adams,3,18001,Unknown,163,108,156,427,264,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,4/26/2021 5:00
4,Adams,3,18001,White,7942,6700,339,14981,7039,12,36,Suppressed,48,36,4/26/2021 5:00


In [22]:

# Data.columns


L= []

for county, record in Data.groupby('county'):
    # print(county)
    D = {}
    D['county'] = county

    for idx, row in record.iterrows():
        d = row.to_dict()
        # print(d)
        race = d['item']
        try:
            D[race + '_' + 'AtLeastOneDose'] = float(d['all_doses_administered'])
        except:
            D[race + '_' + 'AtLeastOneDose'] = np.nan
    # print(D)
    L.append(D)

Indiana_County_Vaccine_Race = pd.DataFrame(L)



In [23]:
print(Indiana_County_Vaccine_Race.shape)
Indiana_County_Vaccine_Race.head() # 95 counties in Pennsylvania

(94, 8)


,county,Asian_AtLeastOneDose,Black or African American_AtLeastOneDose,Other Race_AtLeastOneDose,Unknown_AtLeastOneDose,White_AtLeastOneDose,Hispanic or Latino_AtLeastOneDose,Not Hispanic or Latino_AtLeastOneDose
0,Adams,74.0,31.0,342.0,1061.0,14981.0,395.0,14399.0
1,Allen,7078.0,14018.0,9807.0,25280.0,176402.0,6574.0,184444.0
2,Bartholomew,3386.0,703.0,1619.0,1967.0,46784.0,1721.0,50496.0
3,Benton,10.0,20.0,120.0,180.0,3592.0,161.0,3502.0
4,Blackford,12.0,8.0,53.0,251.0,6162.0,43.0,6083.0


In [24]:
# Indiana_County_Vaccine_Race.to_csv('FinalResult/Indiana.csv')
print(Indiana_County_Vaccine_Race.shape)
Indiana_County_Vaccine_Race.head() # 95 counties in Pennsylvania

(94, 8)


,county,Asian_AtLeastOneDose,Black or African American_AtLeastOneDose,Other Race_AtLeastOneDose,Unknown_AtLeastOneDose,White_AtLeastOneDose,Hispanic or Latino_AtLeastOneDose,Not Hispanic or Latino_AtLeastOneDose
0,Adams,74.0,31.0,342.0,1061.0,14981.0,395.0,14399.0
1,Allen,7078.0,14018.0,9807.0,25280.0,176402.0,6574.0,184444.0
2,Bartholomew,3386.0,703.0,1619.0,1967.0,46784.0,1721.0,50496.0
3,Benton,10.0,20.0,120.0,180.0,3592.0,161.0,3502.0
4,Blackford,12.0,8.0,53.0,251.0,6162.0,43.0,6083.0


In [25]:
# print(len(L))

import numpy as np

state = 'Indiana' ## 

new_L = []


for idx, row in Indiana_County_Vaccine_Race.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['county']) != str:
        print(d)
        continue 
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['county'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Lagrange': 'LaGrange',
        'La Porte': 'LaPorte', 
        'De Kalb': 'DeKalb',
        
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    new_d['Numb_White'] = float(d['White_AtLeastOneDose'])
    new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Black or African American_AtLeastOneDose']
    new_d['Rate_Black'] = np.nan
    
    new_d['Numb_Asian'] = d['Asian_AtLeastOneDose']
    
    new_d['Numb_NaHis'] = d['Not Hispanic or Latino_AtLeastOneDose']
    new_d['Rate_NaHis'] = np.nan
    
    new_d['Numb_Hispa'] = d['Hispanic or Latino_AtLeastOneDose']
    new_d['Rate_Hispa'] = np.nan
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)

not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()


Not included Counties ['Out Of State, Indiana', 'Unknown, Indiana']
(92, 26)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_Hispa,Numb_NaHis,Numb_White,Rate_Asian,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Indiana,Adams,"Adams, Indiana",NaN,74.0,31.0,395.0,14399.0,14981.0,0.540146,...,0.424391,117,208,137,183,198,384,34995,35300,35777
1,Indiana,Allen,"Allen, Indiana",NaN,7078.0,14018.0,6574.0,184444.0,176402.0,0.406058,...,0.562497,1780,4346,17431,19785,45450,53961,302000,313605,379299
2,Indiana,Bartholomew,"Bartholomew, Indiana",NaN,3386.0,703.0,1721.0,50496.0,46784.0,0.474695,...,0.630963,390,806,7133,7427,1974,2756,72810,74147,83779
3,Indiana,Benton,"Benton, Indiana",NaN,10.0,20.0,161.0,3502.0,3592.0,0.476190,...,0.417917,28,69,21,31,97,165,8479,8595,8748
4,Indiana,Blackford,"Blackford, Indiana",NaN,12.0,8.0,43.0,6083.0,6162.0,0.179104,...,0.533045,31,111,67,98,90,172,11378,11560,11758


In [26]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
# STATE_VACCINE = {}
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

T

['Illinois', 'Texas', 'Pennsylvania', 'Indiana']


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_Hispa,Numb_NaHis,Numb_White,Rate_Asian,...,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole,Rate_Parity
0,Indiana,Adams,"Adams, Indiana",NaN,74.0,31.0,395.0,14399.0,14981.0,0.540146,...,117,208,137,183,198,384,34995,35300,35777,0.271524
1,Indiana,Allen,"Allen, Indiana",NaN,7078.0,14018.0,6574.0,184444.0,176402.0,0.406058,...,1780,4346,17431,19785,45450,53961,302000,313605,379299,0.275686
2,Indiana,Bartholomew,"Bartholomew, Indiana",NaN,3386.0,703.0,1721.0,50496.0,46784.0,0.474695,...,390,806,7133,7427,1974,2756,72810,74147,83779,0.286419
3,Indiana,Benton,"Benton, Indiana",NaN,10.0,20.0,161.0,3502.0,3592.0,0.476190,...,28,69,21,31,97,165,8479,8595,8748,0.217449
4,Indiana,Blackford,"Blackford, Indiana",NaN,12.0,8.0,43.0,6083.0,6162.0,0.179104,...,31,111,67,98,90,172,11378,11560,11758,0.452683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Indiana,Washington,"Washington, Indiana",NaN,39.0,24.0,76.0,12103.0,12476.0,0.458824,...,77,208,85,157,143,246,27434,27718,28036,0.286932
88,Indiana,Wayne,"Wayne, Indiana",NaN,464.0,1424.0,744.0,35588.0,33776.0,0.650771,...,252,673,713,1041,3316,4618,59598,61394,65884,0.137297
89,Indiana,Wells,"Wells, Indiana",NaN,62.0,42.0,162.0,13061.0,13674.0,0.328042,...,84,201,189,248,249,429,27415,27754,28296,0.330103
90,Indiana,White,"White, Indiana",NaN,50.0,24.0,407.0,14378.0,14758.0,0.367647,...,176,327,136,202,217,333,23236,23549,24102,0.524536


In [27]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania', 'Indiana']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,90.0,0.28117,0.170887,0.040268,0.169006,0.257110,0.346872,1.313316
1,Rate_White,92.0,0.54505,0.107569,0.278188,0.470123,0.548738,0.610128,0.862574
2,Rate_Parity,90.0,0.26716,0.148162,-0.770827,0.220419,0.275455,0.333261,0.527942


## B. Virginia* (Done)

https://data.virginia.gov/Government/VDH-COVID-19-PublicUseDataset-Vaccines-DosesAdmini/u5ru-3khs

In [28]:
import requests
from  sodapy import Socrata


# Today_Date = str(datetime.now().date())

##########################################
report_date = Today_Date + 'T00:00:00.000'
##########################################

client = Socrata("data.virginia.gov", None)

url = 'https://data.virginia.gov/resource/u5ru-3khs.json?report_date=' + report_date + 'T00:00:00.000?limit=3000'

results = client.get("u5ru-3khs", limit=5000, report_date = report_date)

# Convert to pandas DataFrame
RawData = pd.DataFrame.from_records(results)

In [29]:
while len(RawData) == 0:
    print('Error in Date, Please Change Date:')
    ##########################################
    new_date_for_virgina = input()
    report_date = new_date_for_virgina + 'T00:00:00.000'
    ##########################################

    client = Socrata("data.virginia.gov", None)

    url = 'https://data.virginia.gov/resource/u5ru-3khs.json?report_date=' + report_date + 'T00:00:00.000?limit=3000'

    results = client.get("u5ru-3khs", limit=5000, report_date = report_date)

    # Convert to pandas DataFrame
    RawData = pd.DataFrame.from_records(results)
    
RawData.head()

,report_date,fips,locality,health_district,health_region,vaccination_status,race_and_ethnicity,people_by_vaccination_status_count
0,2021-04-26T00:00:00.000,51045,Craig,Alleghany,Southwest,Fully Vaccinated,Asian or Pacific Islander,5
1,2021-04-26T00:00:00.000,51153,Prince William,Prince William,Northern,Fully Vaccinated,White,34335
2,2021-04-26T00:00:00.000,51570,Colonial Heights,Chesterfield,Central,Not Reported,Black,0
3,2021-04-26T00:00:00.000,51570,Colonial Heights,Chesterfield,Central,Not Reported,Latino,0
4,2021-04-26T00:00:00.000,51001,Accomack,Eastern Shore,Eastern,Not Reported,White,0


In [30]:
RawDataGroups = RawData.groupby('locality')


L = []
for county, record in RawDataGroups:
    D = {}

    D['County'] = county

    for idx, row in record.iterrows():
        d = row.to_dict()
        race = d['race_and_ethnicity'] 
        
        d = row.to_dict()
        # print(d)
        D[race + '_' + d['vaccination_status']] = float(d['people_by_vaccination_status_count'])
        

    L.append(D)

    
Virginia_County_Vaccine_Race = pd.DataFrame(L)

In [31]:
# Virginia_County_Vaccine_Race.to_csv('FinalResult/Vriginia.csv')
print(Virginia_County_Vaccine_Race.shape)
Virginia_County_Vaccine_Race.head() # 67 counties in Pennsylvania

(135, 25)


,County,White_Not Reported,White_Fully Vaccinated,Latino_At Least One Dose,Black_At Least One Dose,Other Race_Not Reported,Native American_At Least One Dose,Native American_Fully Vaccinated,Latino_Fully Vaccinated,Black_Fully Vaccinated,...,Not Reported_Fully Vaccinated,Other Race_Fully Vaccinated,Two or more races_Not Reported,Asian or Pacific Islander_Fully Vaccinated,Latino_Not Reported,Black_Not Reported,Native American_Not Reported,Not Reported_Not Reported,Asian or Pacific Islander_Not Reported,Two or more races_Fully Vaccinated
0,Accomack,0.0,5767.0,542.0,2162.0,0.0,23.0,17.0,369.0,1694.0,...,2041.0,21.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Albemarle,0.0,24795.0,2581.0,3148.0,0.0,214.0,136.0,1530.0,2197.0,...,9198.0,1802.0,0.0,807.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alexandria,0.0,12148.0,7438.0,3818.0,0.0,241.0,119.0,4253.0,2575.0,...,19206.0,2345.0,0.0,923.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alleghany,0.0,2176.0,90.0,181.0,0.0,10.0,7.0,51.0,118.0,...,1520.0,45.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Amelia,0.0,1515.0,79.0,804.0,0.0,17.0,13.0,44.0,599.0,...,662.0,23.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Virginia_County_Vaccine_Race.to_csv('FinalResult/Vriginia.csv')
print(Virginia_County_Vaccine_Race.shape)
Virginia_County_Vaccine_Race.head() # 67 counties in Pennsylvania

(135, 25)


,County,White_Not Reported,White_Fully Vaccinated,Latino_At Least One Dose,Black_At Least One Dose,Other Race_Not Reported,Native American_At Least One Dose,Native American_Fully Vaccinated,Latino_Fully Vaccinated,Black_Fully Vaccinated,...,Not Reported_Fully Vaccinated,Other Race_Fully Vaccinated,Two or more races_Not Reported,Asian or Pacific Islander_Fully Vaccinated,Latino_Not Reported,Black_Not Reported,Native American_Not Reported,Not Reported_Not Reported,Asian or Pacific Islander_Not Reported,Two or more races_Fully Vaccinated
0,Accomack,0.0,5767.0,542.0,2162.0,0.0,23.0,17.0,369.0,1694.0,...,2041.0,21.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Albemarle,0.0,24795.0,2581.0,3148.0,0.0,214.0,136.0,1530.0,2197.0,...,9198.0,1802.0,0.0,807.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alexandria,0.0,12148.0,7438.0,3818.0,0.0,241.0,119.0,4253.0,2575.0,...,19206.0,2345.0,0.0,923.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alleghany,0.0,2176.0,90.0,181.0,0.0,10.0,7.0,51.0,118.0,...,1520.0,45.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Amelia,0.0,1515.0,79.0,804.0,0.0,17.0,13.0,44.0,599.0,...,662.0,23.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
Virginia_County_Vaccine_Race['County'].values

array(['Accomack', 'Albemarle', 'Alexandria', 'Alleghany', 'Amelia',
       'Amherst', 'Appomattox', 'Arlington', 'Augusta', 'Bath', 'Bedford',
       'Bland', 'Botetourt', 'Bristol', 'Brunswick', 'Buchanan',
       'Buckingham', 'Buena Vista City', 'Campbell', 'Caroline',
       'Carroll', 'Charles City', 'Charlotte', 'Charlottesville',
       'Chesapeake', 'Chesterfield', 'Clarke', 'Colonial Heights',
       'Covington', 'Craig', 'Culpeper', 'Cumberland', 'Danville',
       'Dickenson', 'Dinwiddie', 'Emporia', 'Essex', 'Fairfax',
       'Fairfax City', 'Falls Church', 'Fauquier', 'Floyd', 'Fluvanna',
       'Franklin City', 'Franklin County', 'Frederick', 'Fredericksburg',
       'Galax', 'Giles', 'Gloucester', 'Goochland', 'Grayson', 'Greene',
       'Greensville', 'Halifax', 'Hampton', 'Hanover', 'Harrisonburg',
       'Henrico', 'Henry', 'Highland', 'Hopewell', 'Isle of Wight',
       'James City', 'King George', 'King William', 'King and Queen',
       'Lancaster', 'Lee', 'Lexing

In [34]:
# print(len(L))

import numpy as np


virginia_name_dict = {
 'Colonial Heights, Virginia': ['Colonial Heights city, Virginia'],
 
 'Portsmouth, Virginia': ['Portsmouth city, Virginia'],
    
 'Roanoke City, Virginia': ['Roanoke city, Virginia'],
    
 'Charlottesville, Virginia': ['Charlottesville city, Virginia'],
    
 'Waynesboro, Virginia': ['Waynesboro city, Virginia'],
    
 'Virginia Beach, Virginia': ['Virginia Beach city, Virginia'],
    
 'Suffolk, Virginia': ['Suffolk city, Virginia'],
    
 'Williamsburg, Virginia': ['Williamsburg city, Virginia'],
    
 'Roanoke County, Virginia': ['Roanoke, Virginia'],
    
 'Falls Church, Virginia': ['Falls Church city, Virginia'],
    
 'Franklin County, Virginia': ['Franklin, Virginia'],
    
 'Buena Vista City, Virginia': ['Buena Vista city, Virginia'],
 
 'Lynchburg, Virginia': ['Lynchburg city, Virginia'],
    
 'Hampton, Virginia': ['Hampton city, Virginia'],
    
 'Fredericksburg, Virginia': ['Fredericksburg city, Virginia'],
    
 'Staunton, Virginia': ['Staunton city, Virginia'],
    
 'Galax, Virginia': ['Galax city, Virginia'],
    
 'Richmond County, Virginia': ['Richmond, Virginia'],
    
 'Petersburg, Virginia': ['Petersburg city, Virginia'],
    
 'Harrisonburg, Virginia': ['Harrisonburg city, Virginia'],
    
 'Richmond City, Virginia': ['Richmond city, Virginia'],
    
 'Covington, Virginia': ['Covington city, Virginia'],
    
 'Norfolk, Virginia': ['Norfolk city, Virginia'],
    
 'Fairfax City, Virginia': ['Fairfax city, Virginia'],
    
 'Bristol, Virginia': ['Bristol city, Virginia'],
    
 'Isle Of Wight, Virginia': ['Isle of Wight, Virginia'],
    
 'Lexington, Virginia': ['Lexington city, Virginia'],
    
 'Hopewell, Virginia': ['Hopewell city, Virginia'],
    
 'Alexandria, Virginia': ['Alexandria city, Virginia'],
    
 'Manassas City, Virginia': ['Manassas city, Virginia'],
    
 'Winchester, Virginia': ['Winchester city, Virginia'],
    
 'Manassas Park, Virginia': ['Manassas Park city, Virginia'],
    
 'Poquoson, Virginia': ['Poquoson city, Virginia'],
    
 'Emporia, Virginia': ['Emporia city, Virginia'],
    
 'Radford, Virginia': ['Radford city, Virginia'],
    
 'Salem, Virginia': ['Salem city, Virginia'],
    
 'Chesapeake, Virginia': ['Chesapeake city, Virginia'],
    
 'King And Queen, Virginia': ['King and Queen, Virginia'],
    
 'Franklin City, Virginia': ['Franklin city, Virginia'],
    
 'Norton, Virginia': ['Norton city, Virginia'],
  
 'Danville, Virginia': ['Danville city, Virginia'],
    
 'Newport News, Virginia': ['Newport News city, Virginia'],
    
 'Martinsville, Virginia': ['Martinsville city, Virginia']}

RenameCounties = {k.replace(', Virginia', ''): v[0].replace(', Virginia', '') for k, v in virginia_name_dict.items()}

state = 'Virginia' ## 

new_L = []


for idx, row in Virginia_County_Vaccine_Race.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['County']) != str:
        print(d)
        continue 
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    
    new_d['Numb_White'] = d['White_At Least One Dose']
    new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Black_At Least One Dose']
    new_d['Rate_Black'] = np.nan
    
    new_d['Numb_Asian'] = d['Asian or Pacific Islander_At Least One Dose']
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)


not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()

    

Not included Counties ['Not Reported, Virginia', 'Out-of-state, Virginia']
(133, 19)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Virginia,Accomack,"Accomack, Virginia",NaN,89.0,2162.0,6707.0,0.346304,0.232373,0.306270,239,447,257,348,9304,9632,21899,22411,32316
1,Virginia,Albemarle,"Albemarle, Virginia",NaN,1219.0,3148.0,33762.0,0.201454,0.296981,0.377702,383,1000,6051,7189,10600,12008,89388,92002,109330
2,Virginia,Alexandria city,"Alexandria city, Virginia",NaN,1532.0,3818.0,16428.0,0.147620,0.105208,0.154502,998,2243,10378,12886,36290,38626,106329,110857,159428
3,Virginia,Alleghany,"Alleghany, Virginia",NaN,20.0,181.0,3033.0,0.434783,0.259312,0.220054,44,119,46,74,698,890,13783,14040,14860
4,Virginia,Amelia,"Amelia, Virginia",NaN,24.0,804.0,2099.0,0.300000,0.299107,0.208856,79,161,80,124,2688,2827,10050,10264,13145


In [35]:
# Virginia	Accomack	Accomack, Virginia	NaN	63.0	1704.0	5764.0	0.245136	0.183147	0.263208	239	447	257
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania', 'Indiana', 'Virginia']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,133.0,0.205769,0.083771,0.061927,0.146479,0.185904,0.246106,0.565217
1,Rate_White,133.0,0.226679,0.066785,0.089289,0.174478,0.219823,0.259002,0.464662
2,Rate_Parity,133.0,0.020910,0.081434,-0.392125,-0.008385,0.030049,0.064322,0.258536


In [36]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania', 'Indiana', 'Virginia']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,133.0,0.205769,0.083771,0.061927,0.146479,0.185904,0.246106,0.565217
1,Rate_White,133.0,0.226679,0.066785,0.089289,0.174478,0.219823,0.259002,0.464662
2,Rate_Parity,133.0,0.020910,0.081434,-0.392125,-0.008385,0.030049,0.064322,0.258536


In [37]:
print([i for i in STATE_VACCINE])

['Illinois', 'Texas', 'Pennsylvania', 'Indiana', 'Virginia']


# All in One

## Read Previous Vaccine Data

In [42]:
# import os
# path = ''

# print('Input Historical STATE_VACCINE')
# print('For example: CountyVaccine/RaceVaccineBackup/STATE_VACCINE_2021-03-23.p')

# L = []
# while True:
    
#     path = input()
    
#     if os.path.isfile(path):
#         print('Path Exists!')
#         L.append(path)
#     elif path != '':
#         print('Wrong Path')
#     else:
#         break

# print('End')

In [43]:
# import pickle

# Historical_Path = {i.split('_')[-1].replace('.p', ''): i for i in L }

# Historical_Data = {}
# for date, path in Historical_Path.items():
#     with open(path, 'rb' ) as f:
#         Historical_Data[date] = pickle.load(f)
    
    
# [i for i in Historical_Data]

## Show Parity for Each State

In [44]:
# ######################################

# # Today_Date = str(datetime.now().date())
# print(Today_Date)
# ######################################


# for state in STATE_VACCINE:
    
#     print(state)
#     cols = ['name',  'date', 'count', 'mean', 'min','50%', 'max']
    
    
    
#     for date, Old_STATE_VACCINE in Historical_Data.items():
#         x = diagnose_Rate(Old_STATE_VACCINE[state], state = state, date = date)
#         print(x[cols])
    
#     x = diagnose_Rate(STATE_VACCINE[state], state = state, date = Today_Date)
    
#     print(x[cols])
#     print('\n')
    
    
# # pd.concat(L).to_csv('compare.csv')

##  Show Some Specials Cases

In [45]:

# DF = pd.concat([v for k,v in STATE_VACCINE.items()]) 

# data_cols = ['State', 'County', 'Geographic Area Name',
#              'Total_Black', 'Total_White',
#              'Numb_Black', 'Numb_White','Rate_Black','Rate_White', 'Total_Whole', 'Rate_Parity']


# DF[DF['Rate_Black'] > 0.8][data_cols]

## Save Data

In [46]:

import pickle 

path = 'CountyVaccine/Workplace/STATE_VACCINE_' + Today_Date + '_Autonomy.p'

while True:
    print('The data will be saved here, do you agree?' )
    print(path)
    print('If yes, please enter: Yes')
    x = input()
    if x =='Yes':
        break


with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)
    
    
print('Successfully Saved!')

The data will be saved here, do you agree?
CountyVaccine/Workplace/STATE_VACCINE_2021-04-26_Autonomy.p
If yes, please enter: Yes


 Yes


Successfully Saved!
